In [1]:
#Import some modules
%matplotlib inline
import numpy as np
import pandas as pd
import utils as ut


#Load in the data
#loans = pd.read_csv('LoanStats3a.csv', skiprows = 1, 
#                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
#                 skipfooter = 2)

df1 = pd.read_csv('LoanStats3a.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2)
df2 = pd.read_csv('LoanStats3b.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)
df3 = pd.read_csv('LoanStats3c.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)
df4 = pd.read_csv('LoanStats3d.csv', skiprows = 1, 
                  parse_dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'],
                 skipfooter = 2, infer_datetime_format=True)

loans = pd.concat([df1, df2, df3, df4], ignore_index = True)

del df1
del df2
del df3
del df4

/Library/Python/2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [13]:
#Begin by cleaning up the data a bit
#Percentages of the different loan statuses
loans.loan_status.value_counts(normalize = True)


Current                                                0.652293
Fully Paid                                             0.261470
Charged Off                                            0.057966
Late (31-120 days)                                     0.014763
In Grace Period                                        0.006518
Late (16-30 days)                                      0.002723
Does not meet the credit policy. Status:Fully Paid     0.002621
Does not meet the credit policy. Status:Charged Off    0.001005
Default                                                0.000634
Does not meet the credit policy. Status:Current        0.000005
dtype: float64

In [2]:
loans.replace(to_replace = 'Does not meet the credit policy. Status:Fully Paid', value = 'Fully Paid', inplace = True)
loans.replace(to_replace = 'Does not meet the credit policy. Status:Charged Off', value = 'Charged Off', inplace = True)
#Assign default to charged off? They seem equivalent from the lenders perspective
loans.replace(to_replace = 'Default', value = 'Charged Off', inplace = True)

loans.loan_status.value_counts(normalize = True)

Current                                            0.652293
Fully Paid                                         0.264091
Charged Off                                        0.059605
Late (31-120 days)                                 0.014763
In Grace Period                                    0.006518
Late (16-30 days)                                  0.002723
Does not meet the credit policy. Status:Current    0.000005
dtype: float64

In [3]:
#Filter out everything but 'fully paid' and 'charged off' and plot
#We're just going to consider those two categories to simplify the analysis
loans = loans[(loans.loan_status == 'Fully Paid') | (loans.loan_status == 'Charged Off')]

In [16]:
loans.columns

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv',
       u'term', u'int_rate', u'installment', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'issue_d', u'loan_status', u'pymnt_plan',
       u'url', u'desc', u'purpose', u'title', u'zip_code', u'addr_state',
       u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc',
       u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'mths_since_last_major_derog', u'policy_code', u'application_type',
       u'annu

Inferential statistics on the various features of this data set. Begin with the numeric fields that have little to no missing data and do not require any processing.

In [17]:
#Check out the effect of annual income
#Get the annual income for the charged off and fully paid categories
#Get the median incomes
inc_paid_med = loans[loans.loan_status == 'Fully Paid'].annual_inc.median()
inc_chrg_med = loans[loans.loan_status == 'Charged Off'].annual_inc.median()

loans.groupby('loan_status').annual_inc.median()

loan_status
Charged Off    56000
Fully Paid     64000
Name: annual_inc, dtype: float64

In [19]:
#Look at the effect size for the incomes
inc_paid_med / inc_chrg_med
#Modest effect size

1.1428571428571428

In [20]:
#Bootstrap the median incomes for the two categories and see if that effect is significant
prct = ut.med_diff_bootstrap(loans[loans.loan_status == 'Fully Paid'].annual_inc,
                             loans[loans.loan_status == 'Charged Off'].annual_inc, 99.5, 0.5)
prct
#Reliable

[7024.9849999999997, 8773.0099999999984]

In [21]:
#Check out the effect of dti
#Get the dti for the charged off and fully paid categories
#Get the median dtis
dti_paid_med = loans[loans.loan_status == 'Fully Paid'].dti.median()
dti_chrg_med = loans[loans.loan_status == 'Charged Off'].dti.median()

loans.groupby('loan_status').dti.median() 

loan_status
Charged Off    18.27
Fully Paid     15.69
Name: dti, dtype: float64

In [22]:
#Look at the effect size for the dtis
dti_chrg_med / dti_paid_med
#Modest effect size

1.1644359464627152

In [23]:
#Bootstrap the dti for the two categories and see if that effect is significant
prct = ut.med_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].dti,
                             loans[loans.loan_status == 'Fully Paid'].dti, 99.5, 0.5)
prct
#Reliable

[2.4400000000000013, 2.7000000000000011]

In [24]:
#Check out the effect of delinquencies in the last two years
#Get the number of delinquencies for the charged off and fully paid categories
#Get the mean delinquencies
dlq_paid_mn = loans[loans.loan_status == 'Fully Paid'].delinq_2yrs.mean()
dlq_chrg_mn = loans[loans.loan_status == 'Charged Off'].delinq_2yrs.mean()

loans.groupby('loan_status').delinq_2yrs.mean() 

loan_status
Charged Off    0.27490
Fully Paid     0.24146
Name: delinq_2yrs, dtype: float64

In [26]:
#Look at the effect size for the delinquencies
dlq_chrg_mn / dlq_paid_mn
#Modest effect size

1.1384884893215346

In [27]:
#Bootstrap the delinquencies for the two categories and see if that effect is significant
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].delinq_2yrs,
                              loans[loans.loan_status == 'Fully Paid'].delinq_2yrs, 99.5, 0.5)
prct
#Reliable

[0.022954457309100756, 0.043943470247482859]

In [28]:
#Check out the effect of inquiries in the last six months
#Get the inquiries for the charged off and fully paid categories
#Get the mean inquiries
inq_paid_mn = loans[loans.loan_status == 'Fully Paid'].inq_last_6mths.mean()
inq_chrg_mn = loans[loans.loan_status == 'Charged Off'].inq_last_6mths.mean()

loans.groupby('loan_status').inq_last_6mths.mean() 

loan_status
Charged Off    1.047217
Fully Paid     0.858365
Name: inq_last_6mths, dtype: float64

In [30]:
#Look at the effect size for the inquiries
inq_chrg_mn / inq_paid_mn
#Moderate effect

1.2200132638223962

In [31]:
#Bootstrap the inquiries for the two categories and see if that effect is significant
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].inq_last_6mths,
                              loans[loans.loan_status == 'Fully Paid'].inq_last_6mths, 99.5, 0.5)
prct
#Reliable

[0.17240098985823846, 0.2063343733412828]

In [32]:
#Check out the effect of number of open accounts
#Get the number of open accounts for the charged off and fully paid categories
#Get the mean number of accounts
opn_paid_mn = loans[loans.loan_status == 'Fully Paid'].open_acc.mean()
opn_chrg_mn = loans[loans.loan_status == 'Charged Off'].open_acc.mean()

loans.groupby('loan_status').open_acc.mean()

loan_status
Charged Off    11.008180
Fully Paid     10.875742
Name: open_acc, dtype: float64

In [33]:
#Look at the effect size for the number of open accounts
opn_paid_mn / opn_chrg_mn
#Small effect

0.98796918047848403

In [34]:
#Bootstrap the open accounts for the two categories and see if that effect is significant
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Fully Paid'].open_acc,
                              loans[loans.loan_status == 'Charged Off'].open_acc, 99.5, 0.5)
prct
#Looks like the effect is reliable, but it is small enough to be suspect

[-0.19623503540697443, -0.066461859835128423]

In [48]:
#Check out the effect of number of derogatory public records
#Get the number of public records for the charged off and fully paid categories
#Get the mean number of derogatory public records
pub_paid_mn = loans[loans.loan_status == 'Fully Paid'].pub_rec.mean()
pub_chrg_mn = loans[loans.loan_status == 'Charged Off'].pub_rec.mean()

loans.groupby('loan_status').pub_rec.mean()


loan_status
Charged Off    0.143180
Fully Paid     0.139283
Name: pub_rec, dtype: float64

In [49]:
#Look at the effect size for the number of derogatory public records
pub_chrg_mn / pub_paid_mn
#Small effect

1.0279786874048025

In [37]:
#Bootstrap the public records for the two categories and see if that effect is significant
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].pub_rec,
                              loans[loans.loan_status == 'Fully Paid'].pub_rec, 99.5, 0.5)
prct
#Not reliable

[-0.0018740016053750276, 0.009818523648975357]

In [38]:
#Check out the effect of revolving balance
#Get the revolving balance for the charged off and fully paid categories
#Get the median revolving balance 
rev_bal_paid_med = loans[loans.loan_status == 'Fully Paid'].revol_bal.median()
rev_bal_chrg_med = loans[loans.loan_status == 'Charged Off'].revol_bal.median()

loans.groupby('loan_status').revol_bal.median()

loan_status
Charged Off    11301.5
Fully Paid     10805.0
Name: revol_bal, dtype: float64

In [39]:
#Effect size
rev_bal_chrg_med / rev_bal_paid_med
#Small effect

1.0459509486348912

In [40]:
#Bootstrap
prct = ut.med_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].revol_bal, 
                             loans[loans.loan_status == 'Fully Paid'].revol_bal, 99.5, 0.5)
prct
#reliable

[352.0, 647.0]

In [42]:
#Total number of accounts
tot_acc_paid_mn = loans[loans.loan_status == 'Fully Paid'].total_acc.mean()
tot_acc_chrg_mn = loans[loans.loan_status == 'Charged Off'].total_acc.mean()

loans.groupby('loan_status').total_acc.mean() 

loan_status
Charged Off    24.141296
Fully Paid     25.138142
Name: total_acc, dtype: float64

In [43]:
#Effect size
tot_acc_paid_mn  / tot_acc_chrg_mn 
#Small effect

1.041292169210503

In [44]:
#Bootstrap
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Fully Paid'].total_acc,
                              loans[loans.loan_status == 'Charged Off'].total_acc, 99.5, 0.5)
prct
#Seems to be reliable, but not a very big effect

[0.8410510951780743, 1.148117810622356]

In [45]:
#Collections in the last 12 months excluding medical collections
col_paid_mn = loans[loans.loan_status == 'Fully Paid'].collections_12_mths_ex_med.mean()
col_chrg_mn = loans[loans.loan_status == 'Charged Off'].collections_12_mths_ex_med.mean()

loans.groupby('loan_status').collections_12_mths_ex_med.mean()

#Field seems to be missing some data

loan_status
Charged Off    0.007741
Fully Paid     0.006107
Name: collections_12_mths_ex_med, dtype: float64

In [46]:
#Effect size
col_chrg_mn  / col_paid_mn
#Moderate effect, but missing data?

1.2675053795187776

In [47]:
#Bootstrap
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].collections_12_mths_ex_med,
                              loans[loans.loan_status == 'Fully Paid'].collections_12_mths_ex_med, 99.5, 0.5)
prct
#Effect is reliable

[0.00044366518691515099, 0.002854148549792756]

In [50]:
#Number of accounts now delinquent
dlq_acc_paid_mn = loans[loans.loan_status == 'Fully Paid'].acc_now_delinq.mean()
dlq_acc_chrg_mn = loans[loans.loan_status == 'Charged Off'].acc_now_delinq.mean()

loans.groupby('loan_status').acc_now_delinq.mean()

loan_status
Charged Off    0.003879
Fully Paid     0.002827
Name: acc_now_delinq, dtype: float64

In [51]:
#Effect size
dlq_acc_chrg_mn / dlq_acc_paid_mn
#Large effect

1.3722424705392382

In [52]:
#Bootstrap
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].acc_now_delinq,
                              loans[loans.loan_status == 'Fully Paid'].acc_now_delinq, 99.5, 0.5)
prct
#Also reliable

[0.00017702472440258958, 0.0019903687465753829]

Categorical variables:
relatively few categories:
home_ownership, purpose 

lots of categories:
addr_state, zip_code

In [4]:
chrg_off_rate = len(loans[loans.loan_status == 'Charged Off'].loan_status) / float(len(loans.loan_status))
chrg_off_rate

0.1841395271001106

In [5]:
#home ownership
#There is one loan with home_ownwership == 'ANY', add that to 'OTHER'
loans.home_ownership.replace(to_replace = 'ANY', value = 'OTHER', inplace = True)
#Print the fully paid and charged off rates for each ownership category
loans.groupby('home_ownership').loan_status.value_counts(normalize = True).unstack()

,Fully Paid,Charged Off
home_ownership,,
MORTGAGE,0.835505,0.164495
NONE,0.829787,0.170213
OTHER,0.787709,0.212291
OWN,0.811723,0.188277
RENT,0.793776,0.206224


In [10]:
#is_mort
eff_size = (chrg_off_rate / 
    loans[loans.home_ownership == 'MORTGAGE'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Modest effect of is_mort

1.1194209091762977

In [11]:
prct = ut.mean_bootstrap(
    loans[loans.home_ownership == 'MORTGAGE'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is reliable

[0.161742103211789, 0.16729834061714779]

In [12]:
#is_none
eff_size = (chrg_off_rate / 
    loans[loans.home_ownership == 'NONE'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Modest effect of is_none

1.0818197217131498

In [13]:
prct = ut.mean_bootstrap(
    loans[loans.home_ownership == 'NONE'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is not reliable

[0.042553191489361701, 0.31914893617021278]

In [16]:
#is_other
eff_size = (
    loans[loans.home_ownership == 'OTHER'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_other

1.1528785054274671

In [17]:
prct = ut.mean_bootstrap(
    loans[loans.home_ownership == 'OTHER'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is not reliable

[0.13407821229050279, 0.29608938547486036]

In [8]:
#is_own
eff_size = (
    loans[loans.home_ownership == 'OWN'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Small effect of is_own

1.0224701578129374

In [6]:
prct = ut.mean_bootstrap(
    loans[loans.home_ownership == 'OWN'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is not reliable

[0.18144257237778832, 0.19501661129568107]

In [18]:
#is_rent
eff_size = (
    loans[loans.home_ownership == 'RENT'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_rent

1.1199336806700069

In [19]:
prct = ut.mean_bootstrap(
    loans[loans.home_ownership == 'RENT'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is reliable

[0.20299468559680359, 0.20954075992086582]

In [22]:
#loan purpose
#Print the fully paid and charged off rates for each purpose category
loans.groupby('purpose').loan_status.value_counts(normalize = True).unstack()

,Fully Paid,Charged Off
purpose,,
car,0.873259,0.126741
credit_card,0.839346,0.160654
debt_consolidation,0.808992,0.191008
educational,0.791469,0.208531
home_improvement,0.841063,0.158937
house,0.820905,0.179095
major_purchase,0.856680,0.143320
medical,0.793521,0.206479
moving,0.785607,0.214393


In [23]:
#p_is_car
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'car'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Large effect of is_car

1.4528811039327409

In [24]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'car'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#Effect is reliable

[0.11253481894150417, 0.14150417827298051]

In [25]:
#p_is_credit
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'credit_card'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Modest effect of is_credit

1.1461892165678129

In [26]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'credit_card'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct

[0.15646924117830749, 0.16508795669824086]

In [27]:
#p_is_debt
eff_size = (
    loans[loans.purpose == 'debt_consolidation'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Small effect of is_debt

1.0372996019357483

In [28]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'debt_consolidation'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable, but so small it probably doesn't matter much

[0.18837205318940686, 0.19374143572247546]

In [29]:
#p_is_edu
eff_size = (
    loans[loans.purpose == 'educational'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_edu

1.1324608516770678

In [30]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'educational'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#not reliable

[0.15876777251184834, 0.26067535545023507]

In [31]:
#p_is_home_imp
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'home_improvement'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Modest effect of is_home_imp

1.1585678452803312

In [32]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'home_improvement'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable

[0.15113871635610765, 0.16687370600414078]

In [33]:
#p_is_house
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'house'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Small effect of is_house

1.0281647315214366

In [34]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'house'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#not reliable

[0.15586491442542788, 0.20415647921760391]

In [35]:
#p_is_major
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'major_purchase'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#Large effect of is_major

1.2848106370514505

In [36]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'major_purchase'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct

[0.13197146562905318, 0.15466926070038911]

In [37]:
#p_is_med
eff_size = (
    loans[loans.purpose == 'medical'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_med

1.1213191286792417

In [38]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'medical'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable

[0.18689925240299038, 0.22641509433962265]

In [39]:
#p_is_moving
eff_size = (
    loans[loans.purpose == 'moving'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_moving

1.1642953958583948

In [40]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'moving'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable

[0.19140429785107446, 0.23888055972013994]

In [41]:
#p_is_other
eff_size = (
    loans[loans.purpose == 'other'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_other

1.1510439902927163

In [42]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'other'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable

[0.2032943826552161, 0.22089258059974659]

In [43]:
#p_is_renew
eff_size = (
    loans[loans.purpose == 'renewable_energy'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Modest effect of is_renew

1.127119055206893

In [44]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'renewable_energy'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#not reliable

[0.14339622641509434, 0.27547169811320754]

In [45]:
#p_is_small_bus
eff_size = (
    loans[loans.purpose == 'small_business'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean() /
    chrg_off_rate)
eff_size
#Large effect of small business

1.6138188872884882

In [46]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'small_business'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#reliable

[0.28050812161599331, 0.31424406497292795]

In [47]:
#p_is_vac
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'vacation'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#small effect of vacation

1.0369609135601119

In [48]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'vacation'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#not reliable

[0.15165262475696695, 0.20220349967595594]

In [49]:
#p_is_wed
eff_size = (chrg_off_rate /
    loans[loans.purpose == 'wedding'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]).mean())
eff_size
#large effect of wedding

1.3268682169379811

In [50]:
prct = ut.mean_bootstrap(
    loans[loans.purpose == 'wedding'].loan_status.replace(['Fully Paid', 'Charged Off'], value = [0, 1]),
    99.5, 0.5)
prct
#not reliable

[0.1192384769539078, 0.15881763527054107]

Effects of categorical variables requiring minimal processing:
Reliable:
home_ownership: MORTGAGE, RENT
purpose: car, credit_card, debt_consolidation (very small effect), home_improvement, major_purchase, medical,
moving, other, small_business, wedding

Not Reliable:
home_ownership: OWN, NONE, OTHER
purpose: educational, house, renewable_energy, vacation

Numeric fields that require processing:
emp_length, mths_since_last_delinq, mths_since_last_record, revol_util, mths_since_last_major_derog, total_credit_rv

In [53]:
#emp_length
#Create a dictionary to change the year strings into numbers
year_dict = {'emp_length':{'10+ years':10, '< 1 year':0, '3 years':3, '9 years':9, '4 years':4, '5 years':5,
       '1 year':1, '6 years':6, '2 years':2, '7 years':7, '8 years':8, 'n/a':np.nan}}
#Replace the year strings with numbers
loans.replace(year_dict, inplace = True)

In [56]:
#employment length
emp_paid_mn = loans[loans.loan_status == 'Fully Paid'].emp_length.mean()
emp_chrg_mn = loans[loans.loan_status == 'Charged Off'].emp_length.mean()

loans.groupby('loan_status').emp_length.mean()

loan_status
Charged Off    5.773641
Fully Paid     5.806348
Name: emp_length, dtype: float64

In [57]:
#Effect size
emp_paid_mn / emp_chrg_mn
#Very small effect

1.0056649621395004

In [58]:
#Bootstrap
prct = ut.mean_diff_bootstrap(loans[loans.loan_status == 'Charged Off'].emp_length,
                              loans[loans.loan_status == 'Fully Paid'].emp_length, 99.5, 0.5)
prct
#Not reliable

[-0.082100414260775412, 0.018788161397101412]

Imputation protocol:
variables to use for imputation: emp_length, home_ownership, annual_inc, purpose, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, open_acc, pub_rec, revol_bal, total_acc, last_credit_pull_d
*add any zip_code based features, features from NLP on desc*
variables to impute: mths_since_last_delinq, mths_since_last_record, revol_util, mths_since_last_major_derog, total_credit_rv, acc_now_delinq (before 12/10/15)

Numeric fields requiring processing.


Not reliable: emp_length